In [1]:
import random
from typing import List, Tuple

In [2]:
class GeneticAlgorithm:
    def __init__(self, population_size: int, generations: int, mutation_rate: float, items: List[Tuple[int, List[int]]], budgets: List[int]):
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.items = items
        self.budgets = budgets
        self.population = self.create_population(population_size, len(items))
 
    def create_population(self, size: int, n_items: int) -> List[List[int]]:
        return [[random.randint(0, 1) for _ in range(n_items)] for _ in range(size)]
 
    def fitness(self, chromosome: List[int]) -> int:
        total_utility = sum(chromosome[i] * self.items[i][0] for i in range(len(chromosome)))
        total_costs = [sum(chromosome[i] * self.items[i][1][j] for i in range(len(chromosome))) for j in range(len(self.budgets))]
        for cost, budget in zip(total_costs, self.budgets):
            if cost > budget:
                return 0  # Penalize if any constraint is violated
        return total_utility
 
    def selection(self) -> List[int]:
        total_fitness = sum(self.fitness(chromosome) for chromosome in self.population)
        probabilities = [self.fitness(chromosome) / total_fitness for chromosome in self.population]
        return self.population[random.choices(range(len(self.population)), probabilities)[0]]
 
    def crossover(self, parent1: List[int], parent2: List[int]) -> Tuple[List[int], List[int]]:
        point = random.randint(1, len(parent1) - 1)
        return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]
 
    def mutate(self, chromosome: List[int]) -> List[int]:
        for i in range(len(chromosome)):
            if random.random() < self.mutation_rate:
                chromosome[i] = 1 - chromosome[i]
        return chromosome
 
    def repair(self, chromosome: List[int]) -> List[int]:
        total_costs = [sum(chromosome[i] * self.items[i][1][j] for i in range(len(chromosome))) for j in range(len(self.budgets))]
        ordered_items = sorted(range(len(chromosome)), key=lambda x: self.items[x][0], reverse=True)
        for i in ordered_items:
            if chromosome[i] == 1:
                for j in range(len(self.budgets)):
                    if total_costs[j] > self.budgets[j]:
                        chromosome[i] = 0
                        total_costs = [sum(chromosome[i] * self.items[i][1][j] for i in range(len(chromosome))) for j in range(len(self.budgets))]
                        break
        for i in reversed(ordered_items):
            if chromosome[i] == 0:
                if all(total_costs[j] + self.items[i][1][j] <= self.budgets[j] for j in range(len(self.budgets))):
                    chromosome[i] = 1
                    total_costs = [sum(chromosome[i] * self.items[i][1][j] for i in range(len(chromosome))) for j in range(len(self.budgets))]
        return chromosome
 
    def run(self) -> Tuple[List[int], int, List[int]]:
        for generation in range(self.generations):
            new_population = []
            for _ in range(self.population_size // 2):
                parent1 = self.selection()
                parent2 = self.selection()
                offspring1, offspring2 = self.crossover(parent1, parent2)
                new_population.append(self.repair(self.mutate(offspring1)))
                new_population.append(self.repair(self.mutate(offspring2)))
            self.population = new_population
 
        best_solution = max(self.population, key=self.fitness)
        best_value = self.fitness(best_solution)
        best_costs = [sum(best_solution[i] * self.items[i][1][j] for i in range(len(best_solution))) for j in range(len(self.budgets))]
 
        return best_solution, best_value, best_costs

In [3]:
items = [(3, [2, 3]), (4, [3, 2]), (5, [4, 5]), (8, [5, 6])]
budgets = [5, 6]
ga = GeneticAlgorithm(10, 100, 0.1, items, budgets)
best_solution, best_value, best_costs = ga.run()
 
print(f"Best Solution: {best_solution}")
print(f"Total Value: {best_value}")
print(f"Total Costs: {best_costs}")

Best Solution: [1, 1, 0, 0]
Total Value: 7
Total Costs: [5, 5]
